In [2]:
class BinaryTree:
    def __init__(self, key, leftTree=None, rightTree=None):
        self.key = key
        self.leftTree = leftTree
        self.rightTree = rightTree

    def setKey(self, key):
        self.key = key

    def getKey(self):
        return self.key

    def getLeftTree(self):
        return self.leftTree

    def getRightTree(self):
        return self.rightTree

    def insertLeft(self, key):
        if self.leftTree is None:
            self.leftTree = BinaryTree(key)
        else:
            new_node = BinaryTree(key, leftTree=self.leftTree)
            self.leftTree = new_node

    def insertRight(self, key):
        if self.rightTree is None:
            self.rightTree = BinaryTree(key)
        else:
            new_node = BinaryTree(key, rightTree=self.rightTree)
            self.rightTree = new_node

    def printPreorder(self, level=0):
        print(str(level * '-') + str(self.key))
        if self.leftTree is not None:
            self.leftTree.printPreorder(level + 1)
        if self.rightTree is not None:
            self.rightTree.printPreorder(level + 1)
            
    def printInorder(self, level=0):
        if self.leftTree is not None:
            self.leftTree.printInorder(level + 1)
        print(str(level * '-') + str(self.key))
        if self.rightTree is not None:
            self.rightTree.printInorder(level + 1)

    def printPostorder(self, level=0):
        if self.leftTree is not None:
            self.leftTree.printPostorder(level + 1)
        if self.rightTree is not None:
            self.rightTree.printPostorder(level + 1)
        print(str(level * '-') + str(self.key))


class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        if not self.isEmpty():
            return self.items.pop()
        else:
            raise IndexError("Pop from an empty stack")

    def peek(self):
        if not self.isEmpty():
            return self.items[-1]
        else:
            raise IndexError("Peek from an empty stack")

    def size(self):
        return len(self.items)


class Node:
    def __init__(self, key, item):
        self.key = key
        self.item = item
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, key, item):
        new_node = Node(key, item)
        if self.head is None:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node

    def find(self, key):
        current = self.head
        while current:
            if current.key == key:
                return current.item
            current = current.next
        return None

    def sort(self):
        if self.head is None or self.head.next is None:
            return

        size = 1
        current = self.head
        while current.next:
            size += 1
            current = current.next

        step = 1
        while step < size:
            self.head = self.merge_pass(self.head, step)
            step *= 2

    def merge_pass(self, head, step):
        current = head
        prev_tail = None
        new_head = None

        while current:
            left = current
            right = self.split(left, step)
            current = self.split(right, step)

            merged = self.sorted_merge(left, right)

            if prev_tail:
                prev_tail.next = merged
            else:
                new_head = merged

            while prev_tail and prev_tail.next:
                prev_tail = prev_tail.next

        return new_head

    def split(self, head, step):
        if head is None:
            return None
        for _ in range(step - 1):
            if head.next is None:
                break
            head = head.next
        next_head = head.next
        head.next = None
        return next_head

    def sorted_merge(self, a, b):
        if a is None:
            return b
        if b is None:
            return a

        if a.key <= b.key:
            result = a
            result.next = self.sorted_merge(a.next, b)
        else:
            result = b
            result.next = self.sorted_merge(a, b.next)
        return result

In [3]:

def isvariable(char):
    # Define criteria for what you consider a variable
    return char.isalpha()

def isoperator(char):
    # Define criteria for what you consider an operator
    return char in {'+', '-', '*', '/', '^'}

def tokenize_inner(exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif exp[i] == '-':
                if (i != 0 and (exp[i - 1]).isdigit() or isvariable(exp[i - 1]) or exp[i - 1] == ')') and ((exp[i + 1]).isdigit() or isvariable(exp[i + 1]) or isoperator(exp[i + 1])):
                    tokens.append(exp[i])
                    i += 1
                else:
                    if (exp[i + 1]).isdigit() or isvariable(exp[i + 1]):
                        tokens.extend(['(', exp[i + 1], '*', '-1', ')'])
                        i += 2
                    elif exp[i + 1] == '(':
                        # Find the matching closing bracket
                        j = i + 1
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            j += 1
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                        # Recursive call for expression inside the brackets
                        inside_tokens = tokenize_inner(exp[i + 2:j])
                        tokens.extend(['(', *inside_tokens, ')', '*', '-1', ')'])
                        i = j + 1
                    else:
                        print("Error")
                        break
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

def tokenize(expression):
    exp = expression.replace(" ", "")
    return tokenize_inner(exp)


In [4]:
expression = "a = 5**-(1 -2)"
tokenize(expression)

['a', '=', '5', '**', '(', '1', '-', '2', ')', '*', '-1', ')']

In [11]:
def sort_equations(equations):
    def add_dependencies(var, graph, sorted_eqs, seen, undefined_vars):
        if var in seen:
            raise ValueError(f"Circular dependency detected involving '{var}'")
        if var not in sorted_eqs:
            if var not in graph:
                undefined_vars.add(var)
                return
            seen.add(var)
            for dep in graph.get(var, []):
                add_dependencies(dep, graph, sorted_eqs, seen, undefined_vars)
            sorted_eqs[var] = eq_dict[var]
            seen.remove(var)

    # Parsing the equations to extract variables and their dependencies
    eq_dict = {}
    dependency_graph = {}
    for eq in equations:
        var, expr = eq.split(" = ")
        deps = set([x for x in expr.split(" ") if x.isalpha() and x != var])
        eq_dict[var] = expr
        dependency_graph[var] = deps

    # Sorting the equations based on dependencies
    sorted_eqs = {}
    seen = set()
    undefined_vars = set()
    for var in dependency_graph:
        add_dependencies(var, dependency_graph, sorted_eqs, seen, undefined_vars)

    # Adding undefined variables at the end
    for var in undefined_vars:
        sorted_eqs[var] = None

    return sorted_eqs

# Test with the updated set of equations
equations = [
    "d = y",
    "b = a + c",
    "a = (3 + (4 * 5))",
    "c = a ** 2"
]

sorted_equations = sort_equations(equations)
sorted_equations



{'d': 'y', 'a': '(3 + (4 * 5))', 'c': 'a ** 2', 'b': 'a + c', 'y': None}